In [70]:
import pandas as pd
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt

In [86]:
#loading in the current enrollment for only the 2024-25 school year
enrollment_filename = input('Please enter the file name for the file with current enrollment by planning unit by year and grade: ')
current_enrollment = gpd.read_file(f'../DPS-Planning-Final/data/{enrollment_filename}').rename(columns={'pu_2324_848':'pu_2324_84'})
current_enrollment = current_enrollment[['pu_2324_84','grade','fall_year','basez']].replace('', 0)
current_enrollment[['pu_2324_84','grade','fall_year','basez']] = current_enrollment[['pu_2324_84','grade','fall_year','basez']].astype(int)

Please enter the file name for the file with current enrollment by planning unit by year and grade:  current_enrollment.csv


In [97]:
school_type = input('Would you like to get the enrollment projections for elementary, middle, or high schools? Enter as es, ms, or hs: ')

if school_type == 'es':
    grades = [0,1,2,3,4,5]
elif school_type == 'ms':
    grades = [6,7,8]
elif school_type == 'hs':
    grades = [9,10,11,12]

current_by_type = current_enrollment[
    (current_enrollment['grade'].isin(grades)) &
    (current_enrollment['fall_year'].isin([2022,2023,2024]))
     ]

averaged_by_type = current_by_type.groupby(['pu_2324_84','grade'],as_index=False).mean()
averaged_by_type = averaged_by_type.groupby(['pu_2324_84'],as_index=False).sum().drop(columns=['grade','fall_year'])

all_pus = pd.DataFrame({'pu_2324_84': range(1,852)})
full_basez = all_pus.merge(averaged_by_type, on='pu_2324_84',how='left').fillna(0)

full_basez.loc[773,'basez'] = full_basez.loc[773,'basez'] * 30.0/81.0
full_basez.loc[850,'basez'] = full_basez.loc[773,'basez'] * 51.0/81.0
full_basez[['basez']] = full_basez[['basez']].map(lambda x: int(x))
full_basez

Would you like to get the enrollment projections for elementary, middle, or high schools? Enter as es, ms, or hs:  es


,pu_2324_84,basez
0,1,7
1,2,4
2,3,2
3,4,7
4,5,7
...,...,...
846,847,5
847,848,12
848,849,0
849,850,0


In [102]:
#read in the planning units
pu_filename = input('Please input the geodataframe with all planning units: ')
dps_pu = gpd.read_file(f'../DPS-Planning-Final/data/{pu_filename}').rename(columns={'pu_2324_848':'pu_2324_84'})
dps_pu = dps_pu.to_crs(epsg = 3857).sort_values(by='pu_2324_84')
dps_pu

Please input the geodataframe with all planning units:  pu_2324_SPLIT.geojson


,OBJECTID,pu_2324_84,X,Y,M_min,PS_ID,PUID2122_2,ps_id_833,psid_982,TIMS_PU,Region,Shape_Length,Shape_Area,geometry
0,1,1,2.049028e+06,893128.606346,0.0,589,589,1,1,PU589,North,26532.120021,3.556198e+07,"POLYGON ((-8.77e+06 4.33e+06, -8.77e+06 4.33e+..."
1,2,2,2.039923e+06,835434.121317,0.0,401,401,2,2,PU401,North,19333.085511,2.080136e+07,"POLYGON ((-8.78e+06 4.31e+06, -8.78e+06 4.31e+..."
2,3,3,2.058005e+06,797649.028742,0.0,782,782.0,3,3,PU782.0,East,18314.823119,1.329077e+07,"POLYGON ((-8.77e+06 4.29e+06, -8.77e+06 4.29e+..."
3,4,4,2.003222e+06,772895.737307,0.0,201,201.0,4,4,PU201.0,Southwest,29282.100083,4.684140e+07,"POLYGON ((-8.79e+06 4.29e+06, -8.79e+06 4.29e+..."
4,5,5,2.067334e+06,828244.230730,0.0,646,646,6,6,PU646,East,60685.151474,1.497046e+08,"POLYGON ((-8.77e+06 4.3e+06, -8.77e+06 4.3e+06..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
848,846,846,2.016360e+06,858711.587472,0.0,573,573,77,883,PU573,North,16660.241935,9.218197e+06,"POLYGON ((-8.79e+06 4.32e+06, -8.79e+06 4.32e+..."
849,847,847,2.013328e+06,854621.040074,0.0,322,322,50,50,PU322,North,9379.857191,3.748151e+06,"POLYGON ((-8.79e+06 4.31e+06, -8.79e+06 4.31e+..."
850,848,848,2.032661e+06,834741.061398,0.0,511,511,496,496,PU511,North,4334.417303,1.033344e+06,"POLYGON ((-8.78e+06 4.31e+06, -8.78e+06 4.31e+..."
201,849,849,2.040116e+06,820883.548010,0.0,292,292,255,255,PU292,North,35118.764031,2.898398e+07,"POLYGON ((-8.79e+06 4.31e+06, -8.79e+06 4.31e+..."


In [108]:
full_geo = dps_pu.merge(full_basez, on = 'pu_2324_84')[['pu_2324_84'"",'Region','geometry','basez']]
#merge the student counts with the geometries
full_geo
#now we have the student counts and geometries for each planning unit

,pu_2324_84,Region,geometry,basez
0,1,North,"POLYGON ((-8.77e+06 4.33e+06, -8.77e+06 4.33e+...",7
1,2,North,"POLYGON ((-8.78e+06 4.31e+06, -8.78e+06 4.31e+...",4
2,3,East,"POLYGON ((-8.77e+06 4.29e+06, -8.77e+06 4.29e+...",2
3,4,Southwest,"POLYGON ((-8.79e+06 4.29e+06, -8.79e+06 4.29e+...",7
4,5,East,"POLYGON ((-8.77e+06 4.3e+06, -8.77e+06 4.3e+06...",7
...,...,...,...,...
846,846,North,"POLYGON ((-8.79e+06 4.32e+06, -8.79e+06 4.32e+...",0
847,847,North,"POLYGON ((-8.79e+06 4.31e+06, -8.79e+06 4.31e+...",5
848,848,North,"POLYGON ((-8.78e+06 4.31e+06, -8.78e+06 4.31e+...",12
849,849,North,"POLYGON ((-8.79e+06 4.31e+06, -8.79e+06 4.31e+...",0
